In [1]:
import explicit_statetespace as efss
import ssr_kronecker_vector as ssr
import ssr_likelihood as ssr_lklhd
import numpy as np


In [2]:
n = 6
npone = n + 1
sparsity = 0.5
log_theta = np.zeros((npone, npone))
log_theta += np.diag(np.random.normal(size=npone))
index = np.argwhere(log_theta == 0)[
    np.random.choice(npone**2-npone, size=int((npone**2-npone)
                     * (1-sparsity)), replace=True)
]
log_theta[index[:, 0], index[:, 1]] = np.random.normal(
    size=int((npone**2-npone)*(1-sparsity)))


In [3]:
state = np.random.randint(2, size=2*n+1)
length = 2**sum(state)
state


array([0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0])

In [42]:
# Test explicit ssr vs. shuffle trick ssr
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    assert (np.allclose(
        efss.ssr_build_q(dpoint=state, log_theta=log_theta) @ p,
        ssr.kronvec(log_theta=log_theta, p=p, n=n, state=state)
    ))


In [13]:
mut = [state[j] + 2 * state[j + 1] for j in range(0, 2 * n, 2)]
mut.append(state[-1])
mut = np.array(mut)


In [26]:
# Test explicit ssr vs. shuffle trick ssr in sync
for j in range(length):
    p = np.zeros(length)
    p[j] = 1
    for i in range(n):
        assert (np.allclose(
            efss.sync_ssr_q(mut=mut, theta=np.exp(log_theta), i=i, n=n) @ p,
            ssr.kronvec_sync(log_theta=log_theta, p=p, n=n, i=i, state=state)
        ))


In [27]:
# Test explicit ssr vs. shuffle trick ssr in async prim
for j in range(length):
    p = np.zeros(length)
    p[j] = 1
    for i in range(n):
        assert (np.allclose(
            efss.prim_ssr_q(mut=mut, theta=np.exp(log_theta), i=i, n=n) @ p,
            ssr.kronvec_prim(log_theta=log_theta, p=p, n=n, i=i, state=state)
        ))


In [28]:
# Test explicit ssr vs. shuffle trick ssr in async met
for j in range(length):
    p = np.zeros(length)
    p[j] = 1
    for i in range(n):
        assert (np.allclose(
            efss.met_ssr_q(mut=mut, theta=np.exp(log_theta), i=i, n=n) @ p,
            ssr.kronvec_met(log_theta=log_theta, p=p, n=n, i=i, state=state)
        ))


In [29]:
# Test explicit ssr vs. shuffle trick ssr in seeding
for j in range(length):
    p = np.zeros(length)
    p[j] = 1
    assert (np.allclose(
        efss.seeding_ssr_q(mut=mut, theta=np.exp(log_theta), n=n) @ p,
        ssr.kronvec_seed(log_theta=log_theta, p=p, n=n, state=state)
    ))


In [30]:
# Test no diag
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    q = ssr.kronvec(log_theta=log_theta, p=p, n=n, state=state)
    q[j] = 0
    assert (np.allclose(
        q,
        ssr.kronvec(log_theta=log_theta, p=p, n=n, state=state, diag=False)
    ))


In [31]:
# Test diag
assert (np.allclose(
    np.diag(efss.ssr_build_q(dpoint=state, log_theta=log_theta)),
    ssr.kron_diag(log_theta=log_theta, n=n, state=state)
))


In [34]:
# Test R inverse
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    assert (np.allclose(
        np.linalg.inv(np.identity(
            length) - efss.ssr_build_q(dpoint=state, log_theta=log_theta)) @ p,
        ssr_lklhd.R_i_inv_vec(log_theta=log_theta, x=p, lam=1, state=state)
    ))


In [40]:
# Test transpose
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    assert (np.allclose(
        efss.ssr_build_q(dpoint=state, log_theta=log_theta).T @ p,
        ssr.kronvec(log_theta=log_theta, p=p, n=n, state=state, transpose=True)
    ))
